In [ ]:
# a tracker/deterrent based on using a points system 

# structure to hold new items (name, website link, price, category, value, date logged, redeemed?, etc.)
# function to add new items 
# function to redeem item (will have to update the structure and total points)
# function to total up point value in structure
# function to search structure by name, category, value
# functions to calculate points based on current date

# function to show you what's possible to purchase based on point values today


# create item as its own class 
# init to create the base of the object (name, link, price, category, value, date logged, date redeemed, delete_ind, etc.)
# function to update data

# tracker class will create item objects, store them, calculate points, etc. 

# menu class 
# calculate points and set it upon starting
# load most recent saved data
# save the data after every session 


In [ ]:
# structure to hold items (can use pandas dfand maybe pickle the data?)
# could use pickle or json to serialize the data
# error checking for values entered for the wishlist item. maybe that should be handled with the menu?

# define the wishlist categories?
# in the future, create a way to assign the point value?

In [1]:
# import packages 
import datetime
import pickle
import os
import time
from tabulate import tabulate
#from pathlib import Path
from sys import exit

In [2]:
class Tracker():
    
    """
    Tracker object to hold the wishlist items and track point values
    """
    
    def __init__(self, username):
        
        # creates the wishlist dict
        self.__wishlist = dict()
        # captures when the tracker was started
        self.__start_date = datetime.date.today()
        # captures current date for state purposes
        self.__cur_date = datetime.date.today()
        # inital points accumulated
        self.__points = 0
        # initial points value of wishlist
        self.__wl_points = 0
        # initial wishlist cost
        self.__cost = 0
        # get a username
        self.__username = username
        
    def tracker_info(self, debug = "n"):
        
        print("Start Date of Tracker:", self.__start_date)
        print("Username is:", self.__username)
        
        if debug == "y":
            print("Wishlist contains:", self.__wishlist)
        
    
    def is_in_wishlist(self, item):
        
        """
        Function to check whether something is in the wishlist
        """
        
        return item in self.__wishlist.keys()
        
    def print_wishlist_old(self, csv = "no"):
        
        """Function to put the wishlist dict into a df 
        for easy viewing with an option to create csv"""
        
        # have not built in csv functionality
        df = pd.DataFrame(self.__wishlist)
        df_T = df.T
        
        return df_T[(df_T["redeemed"][0] == "n") & (df_T["del_ind"][0] == "n")]
    
    def print_wishlist(self, only_active = "y"):
        
        """
        Function that uses tabulate to create a formatted table for user that 
        either contains only active items or all items regardless of deletion
        or redemption        
        """
        #QUESTION: add sorting to these tables? Like by date?

        formatted_wl = []
        
        
        if only_active == "y":
            
            for k, v in self.__wishlist.items():
                if v["redeemed"][0] == "n" and v["del_ind"][0] == "n":
                    formatted_wl.append((k, v["date"].strftime("%m/%d/%Y"), v["category"], 
                                         float(v["price"]), v["value"]))
            
            headers = ["Item", "Date", "Category", "Price", "Value"]
                    
        elif only_active == "n":
            
            for k, v in self.__wishlist.items():
                formatted_wl.append((k, v["date"].strftime("%m/%d/%Y"), v["category"], 
                                     float(v["price"]), v["value"], v["del_ind"], v["redeemed"]))
                
            headers = ["Item", "Date", "Category", "Price", "Value", "Deleted?", "Redeemed?"]

        return tabulate(formatted_wl, headers = headers)
    
    # getter
    @property 
    def points(self):
        return self.__points
    
    @property
    def wishlist(self):
        return self.__wishlist
    
    @property
    def points(self):
        return self.__points
    
    @property
    def cost(self):
        return self.__cost
    
    @property
    def wl_points(self):
        return self.__wl_points
        
    #@points.setter
    #def set_points(self, points):
        
        #"""After calculating the points for the day, set that as the points """
        
        #self.points = points

    class IsDel(Exception):
        
        """ Raised when item has been deleted to prevent performing actions on deleted items """
        
        def __init__(self, item, message = "Unable to perform action. Item has been deleted."):
            self.item = item
            self.message = message
            super().__init__(self.message)
            
        def __str__(self):
            
            return "{} --> {}".format(self.item, self.message)

    
    class IsRedeemed(Exception):
        
        """ Raised when item has been redeemed to prevent peforming actions on redeemed items"""
        
        def __init__(self, item, message = "Unable to perform action. Item has been redeemed."):
            self.item = item
            self.message = message
            super().__init__(self.message)
            
        def __str__(self):
            
            return "{} --> {}".format(self.item, self.message)
    
    # QUESTION: should I have this class if I already have the function check whether it's in there...?
    class NotInWishlist(Exception):
        
        """ Raised when item is not in wishlist """
        
        def __init__(self, item, message = "Unable to perform action. Item is not in wishlist."):
            self.item = item
            self.message = message
            super().__init__(self.message)
            
        def __str__(self):
            
            return "{} --> {}".format(self.item, self.message)
    
    class NotEnough(Exception):
        
        """ Raised when item redemption exceeds point total"""
        
        def __init__(self, item, message = "Unable to perform action. Item value exceeds accumulated point total."):
            self.item = item
            self.message = message
            super().__init__(self.message)
            
        def __str__(self):
            
            return "{} --> {}".format(self.item, self.message)
        
    def add_item(self, item, price, category, value, 
                          override_dates = "n", date = datetime.date(1900, 1, 1)):
        
        """
        Function to add wishlist item if it doesn't already exist in the dict
        """

        if self.is_in_wishlist(item) is False:
            
            if override_dates == "n":
                self.__wishlist[item] = {"price":price, "category":category, 
                                         "value":value, "date":datetime.date.today(),
                                         "redeemed_dt":datetime.date(1900, 1, 1), "redeemed":"n",
                                        "del_ind": "n"}
                print("Item added to wishlist")
            elif override_dates == "y":
                # allows overriding logging today's date in case we're backtracking
                self.__wishlist[item] = {"price":price, "category":category, 
                                         "value":value, "date":date, 
                                         "redeemed_dt":datetime.date(1900, 1, 1), "redeemed":"n",
                                        "del_ind": "n"}
                print("Item added to wishlist")
                
        else:
            # revisit to create a way to bypass this upon user confirmation
            print("Item is already in wishlist")
            

    
    def del_item(self, item):
        
        """
        Function to mark an item as deleted. We don't do a hard delete in case we need to backtrack.
        """
        # TO DO: prevent delete if it has already been deleted or redeemed using errors
        try:
            
            if self.is_in_wishlist(item) is False:
                raise self.NotInWishlist(item)
            
            if self.__wishlist[item]["del_ind"] == "y":
                raise self.IsDel(item)

            if self.__wishlist[item]["redeemed"] == "y":
                raise self.IsRedeemed(item)
            
            self.__wishlist[item]["del_ind"] = "y"
            print("Item deleted from wishlist")
            
        except self.NotInWishlist:
            print("Item not in wishlist.")
        
        except self.IsDel:
            print("Unable to perform action. Item already has a status of deleted.")
        
        except self.IsRedeemed:
            print("Unable to perform action. Cannot delete a redeemed item.")
            
                
    def update_item(self, item, to_update, update):
        
        """
        Function to update an item on the wishlist based on the key defined
        """
        
        try:
            
            if self.is_in_wishlist(item) is False:
                raise self.NotInWishlist(item)
            
            if self.__wishlist[item]["del_ind"] == "y":
                raise self.IsDel(item)

            if self.__wishlist[item]["redeemed"] == "y":
                raise self.IsRedeemed(item)
                
                
            self.__wishlist[item][to_update] = update
            print("Item has been updated")
            
        except self.NotInWishlist:
            print("Item not in wishlist.")
        
        except self.IsDel:
            print("Unable to perform action. Item already has a status of deleted.")
        
        except self.IsRedeemed:
            print("Unable to perform action. Cannot delete a redeemed item.")
            
    def calc(self):
        
        """
        Function to calculate the total points and cost based on the number of non-redeemed items current in the list
        """
        
        points = 0
        wl_points = 0
        cost = 0
        
        for item in self.__wishlist.keys():
            temp = self.__wishlist[item]
            
            # for items that are active and haven't been redeemed or deleted
            if temp["date"] <= datetime.date.today() and temp["redeemed"][0] == "n" and temp["del_ind"][0] == "n":
                
                points += abs(datetime.date.today() - temp["date"]).days
                wl_points += temp["value"]
                cost += temp["price"]
            
            # for items that have been redeemed, add back the value up until redeemed
            # then subtracted the value redeemed
            if temp["redeemed_dt"] > datetime.date(1900, 1, 1) and temp["redeemed_dt"] <= datetime.date.today() and temp["del_ind"][0] == "n":

                points += abs(temp["redeemed_dt"] - temp["date"]).days
                points -= temp["value"]
        
        # set points attribute
        self.__points = points
        self.__cost = cost
        self.__wl_points = wl_points 
        
    def redeem_item(self, item):
        
        """
        Function to mark an item as redeemed in the wishlist if its point value is not larger
        than the points total, item hasn't been deleted before, item hasn't been redeemed before
        
        Before redeeming:
        - check that item is in wishlist --> otherwise, keyerror or not in wishlist
        - check that item hasn't been deleted --> otherwise, not in wishlist
        - check that item value is less than or equal to current points total --> otherwise, print error and don't redeem
        """
        
        try:

            if self.is_in_wishlist(item) is False:
                raise self.NotInWishlist(item)
            
            if self.__wishlist[item]["del_ind"] == "y":
                raise self.IsDel(item)

            if self.__wishlist[item]["redeemed"] == "y":
                raise self.IsRedeemed(item)

            if self.__wishlist[item]["value"] > self.__points:
                raise self.NotEnough(item)
            
            # QUESTION: do I need to the private reference here if I set up the getter??
            self.__wishlist[item]["redeemed"] = "y"
            self.__wishlist[item]["redeemed_dt"] = datetime.date.today()
            self.__points -= self.__wishlist[item]["value"]
            self.__cost -= self.__wishlist[item]["price"]
            
            print("Item successfully redeemed!")
        
        # QUESTION: do I need all the details in the error classes if I'm just going to print anyway?
        except self.NotInWishlist:
            print("Item not in wishlist.")
        
        except self.IsDel:
            print("Unable to perform action. Item has a status of deleted.")
        
        except self.IsRedeemed:
            print("Unable to perform action. Item has already been redeemed.")
        
        except self.NotEnough:
            print("Unable to perform action. Item value exceeds the accumulated point total!")

    
    def search_wishlist(self, item):
        
        # may have to be broken down into other methods depending on search?
        
        """
        Function to return item dict if item exists
        """
        
        if self.is_in_wishlist(item):
            
            return self.__wishlist[item]
        
        else:
            return False

In [3]:
class Session():
    
    """
    User interface for the Impulse Tracker
    """
    # hold all the tracker names that are created
    tracker_names = set()
    
    def __init__(self):
        self.cur_tracker = None
        self.cur_username = ''
    
    def display_main(self):
        """ 
        Displays the main menu of creating a new tracker, loading previous tracker, or quit 
        """
                
        while True:
            print("""
                     __             __   ___     __   __   ___       __          __  
            |  |\/| |__) |  | |    /__` |__     /__` |__) |__  |\ | |  \ | |\ | / _` 
            |  |  | |    \__/ |___ .__/ |___    .__/ |    |___ | \| |__/ | | \| \__> 

            ___  __        __        ___  __                                         
             |  |__)  /\  /  ` |__/ |__  |__)                                        
             |  |  \ /~~\ \__, |  \ |___ |  \     
            """)

            print("""
            Welcome to Impulse Spending Tracker, where you can create a wishlist and curb
            impulse spending using a point system. The longer items stay un-redeemed on 
            your wishlist, the more points you accumulate. Accumulate enough points and you
            can redeem items from your wishlist. 

            What would you like to do?

            1) Create a new tracker
            2) Load a saved tracker
            3) Quit
            """)

            resp = input("Enter your response here: ")

            if resp == '1':
                self.create_tracker()
                break
            elif resp == '2':
                self.load_tracker()
                break
            elif resp == '3':
                self.quit()
            else:
                print("Invalid response. Please try again.")
    
    def display_post_create(self):
        """
        Displays the menu after you initialize a new tracker with create_tracker
        """
        
        print("""
        To get started...
        
        QUICK START MENU
        
        1) Add an item to the wishlist
        2) Go to main menu
        3) Save and quit
        
        """)
        
        while True:
            
            resp = input("What would you like to do? ")
            
            if resp == '1':
                self.add_item_run()
                self.display_functions()
            elif resp == '2':
                self.display_functions()
            elif resp == '3':
                # have it save by default
                self.save_tracker()
                self.quit()
                break

            
    
    def display_functions(self):
        """
        Displays the functional menu after you either create a new tracker and 
        add at least one item or if you load a saved tracker
        """
          
        while True:
            print("""
            Select one of the following options:

            1) Add an item to the wishlist
            2) Update an item in the wishlist
            3) Delete an item from the wishlist
            4) Redeem an item from the wishlist
            5) Search for a wishlist item by name
            6) View wishlist
            7) View tracker info
            8) Save tracker
            9) Save tracker and quit
            """)
            resp = input("What would you like to do? ")
            
            if resp == "1":
                self.add_item_run()
            elif resp == "2":
                self.update_item()
            elif resp == "3":
                self.del_item()
            elif resp == "4":
                self.redeem_item()
            elif resp == "5":
                self.search_item()
            elif resp == "6":
                self.print_wishlist()
            elif resp == "7":
                self.tracker_info()
            elif resp == "8":
                self.save_tracker()
            elif resp == "9":
                self.quit()
            else:
                print("Invalid response. Please try again.")
    
    def run(self):
        # upon running, if there isn't a data folder created, make one
        # load the tracker dict so later we can check to see if there's a conflict if
        # someone tries to use the same username
        
        if not os.path.isdir("data"):
            os.makedirs("data")
            
        # then check if there's a pickle file for the tracker names
        # if so, load the most recent one and assign to tracker_names
        
        pickles = ["data/" + f for f in os.listdir("data") if f[0:13] == "tracker_names"]
        
        if len(pickles) > 0:

            newest = max(pickles , key = os.path.getctime)
            
            with open(newest, "rb") as f:
                self.tracker_names = pickle.load(f)
        
        self.display_main()
        
    
    def create_tracker(self):
        """
        Creates a new tracker object if username/tracker doesn't already exist
        """
        print("Let's make a new Tracker!")
        
        while True:
            
            username = input("Enter a username to associate with the Tracker: ")
            
            if username in self.tracker_names:
                print("""
                Username already exists in system. 
                
                1) Return to main menu to load tracker associated with that username
                2) Select another username               
                """)
                
                resp = input("Enter a response: ")
                
                if resp == "1":
                    self.display_main()
                elif resp == "2":
                    continue
            
            else:
                break
            
        # initialize Tracker
        tracker = Tracker(username)

        # add tracker to menu's tracker names
        self.tracker_names.add(username)
        # set this new tracker as the current tracker
        self.cur_tracker = tracker
        self.cur_username = username
        print("Tracker successfully created!")
        self.display_post_create()
                  
    
    def load_tracker(self):
        """
        Checks if inputted username is in tracker_names, which is loaded and assigned 
        based on whether a pickle file for the tracker name exists. If does exist, 
        then load the most recent tracker pickle file associated with that name inside
        the data directory
        """
        while True:
            
            # if there are saved trackers to load
            if len(self.tracker_names) > 0:
                
                username = input("Enter the username of the tracker you want to load: ")

                if username not in self.tracker_names:
                    print("""
                    Sorry, there is no saved tracker for that username. 

                    1) Re-enter the username
                    2) Create a new tracker
                    """)

                    resp = input("What would you like to do? ")

                    if resp == '1':
                        continue
                    elif resp == '2':
                        self.create_tracker()
                        break
                    else:
                        print("Invalid response. Try again.")
                else:
                    break
            else:
                print("There are no saved trackers in the system. Please create a new Tracker.")
                self.create_tracker()
                break
                
        
        # find the latest file of the corresponding username
        
        # set current username to username
        self.cur_username = username
        
        # make a list of all the files that correspond to username
        pickles = ["data/" + f for f in os.listdir("data") if f[0:len(username)] == username]
        
        # if there's at least one file, find the most recent one
        # and load it as the current tracker
        
        if len(pickles) > 0:

            newest = max(pickles , key = os.path.getctime)
            
            with open(newest, "rb") as f:
                self.cur_tracker = pickle.load(f)   
        
        # calculate the latest points and cost since last save
        self.cur_tracker.calc()
        print("Tracker successfully loaded!")

        self.display_functions()
            
    def add_item_date(self):
        
        """
        Asks user if they want to override date timestamp
        Checks if the date inputs are correct
        Returns formatted date object for the Tracker.add_item()
        """
        
        while True:
            
            print("""
            When an item is added to the wishlist, it is timestamped to today's date
            by default. You have the option to override this and input a custom date.
            For example, if you were tracking items elsewhere. Note: Item dates are used
            to calculate point accummulation so altering them will affect your points.     
                  """)
            resp = input("Do you want to override the item date? y/n ")
            
            if resp[0].lower() == "y":
                
                while True:
                    str_date = input("Enter a custom item date in the following format (MM/DD/YYYY): ")
                    
                    try:
                        return "y", datetime.datetime.strptime(str_date, "%m/%d/%Y").date()
                    except:
                        print("Invalid date. Please use the MM/DD/YYYY format. Try again.")
                        #continue
                        # QUESTION: add a quit/exit function? or just break so it passes the control
                                        
            elif resp[0].lower() == "n":
                
                return "n", None
            
            else:
                print("Invalid response. Please try again.")
                #continue
                
    def add_item_price(self):
        
        """
        Asks user to enter price of the item
        Performs error checking
        Returns the object
        """
        # user must enter either a float or int price, but internally must be converted to float
        
        while True:
            
            resp = input("Enter the item price $:")
            
            try:
                # no zero dollars or negative numbers
                if float(resp) <= 0:
                    print("Please enter a price that is greater than 0 dollars.")
                    continue
                else:
                    return float(resp)
            
            except:
                # catches if they put non-numerical values
                print("Invalid monetary value. Please try again.")
 
    
    def add_item_cat(self):
        
        """
        Asks user to select a category for the item
        Performs error checking
        Returns the object
        """

        categories = {"1": "Beauty/Skincare", "2":"Books", "3":"Clothing/Accessories", 
                      "4": "Electronics", "5":"Food", "6":"Jewelry", "7":"Other"}
        
        while True:
            
            print("""
            Item Categories:
            
            1) Beauty/Skincare
            2) Books
            3) Clothing/Accessories
            4) Electronics
            5) Food
            6) Jewelry
            7) Other
            
            """)
            
            resp = input("Select an item category by the number: ")
            
            # if valid response then return it
            if resp in [str(i) for i in range(1, 8)]:
                
                return categories[resp]
            else:
                print("Invalid response. Please try again.")

    def add_item_value(self):
        
        """
        Asks user to enter a point value for the item
        Performs error checking (no negatives, nothing too high)
        Returns object
        """
        
        while True:
            
            print('''
            Enter a point value for the item. This is what it will 
            "cost" to redeem the item once you accumulate enough points.
            Point values can range from 100 to 2000 depending on your own
            personal factors such as price, necessity, timing, etc.
            ''')
            
            resp = input("Item point value: ")
            
            if int(resp) >= 100 and int(resp) <= 2000:
                return int(resp)
            else:
                print("Invalid response. Please try again.")
          
    def add_item_run(self):
        """
        Adds an item to the wishlist utilizing Tracker function after
        running the functions to error check inputs to the Tracker.add_item() function
        """
        
        while True:
            
            item = input("To get started with adding an item to the wishlist, enter the item name: ")

            if self.cur_tracker.is_in_wishlist(item):
                print("""
                This item already exists in the wishlist. 
                1) Enter a new item
                2) Update existing item 
                """)
                resp = input("What would you like to do? ")
                
                if resp == "1":
                    continue
                elif resp == "2":
                    #  QUESTION: possible to pass in what item they inputted previously?
                    # would make the transition more seamless
                    self.update_item()
                    self.display_functions()
                else:
                    continue
            else:
                break
                
        try:
            category = self.add_item_cat()
            price = self.add_item_price()
            value = self.add_item_value()
            override, date = self.add_item_date()
            
            self.cur_tracker.add_item(item = item, price = price, category = category, 
                                      value = value, override_dates = override, date = date)
        except:
            print("Something went wrong?")
            
        finally:
            self.display_functions()
            
            
    
    def update_item(self):
        """
        Asks user what item to updated and what part they want to update
        """
        # TO DO: add error checking so that you can't update a deleted item or redeemed item
        
        while True:
            
            resp = input("What is the name of the item you would like to update? ")
            
            if self.cur_tracker.is_in_wishlist(resp):
                # QUESTION: allow updates of the date added?
                
                print("""
                What would you like to update for this item?
                
                1) Category
                2) Price
                3) Point value
                """)
                
                options = {"1":["category", self.add_item_cat], 
                           "2":["price", self.add_item_price], 
                           "3":["value", self.add_item_value]}
                
                resp2 = input("Select one of the options above by typing in the number: ")
                
                if resp2 in options.keys():
                    
                    # run the corresponding function which has error checking
                    resp3 = options[resp2][1]()
                
                self.cur_tracker.update_item(resp, options[resp2][0], resp3)
                
                if resp2 == "2" or resp2 == "3":
                    self.cur_tracker.calc()
                break

            else:
                
                print("That item is not in the wishlist. Please try again.")
                
                # QUESTION: add an opt out?
                       
    
    def del_item(self):
        
        """
        Asks user what item they want to delete, checks if it's in wishlist,
        deletes if in wishlist.
        
        Note: this is not a hard delete. It marks the delete_ind as yes
        """
        
        while True:
            
            # TO DO: make it not as case sensitive
            resp = input("What item do you want to delete from the wishlist? Please type the exact name: ")
            
            if self.cur_tracker.is_in_wishlist(resp):
                
                resp2 = input("Please type Y to confirm that you want to delete this item: ")
                
                if resp2.lower() == 'y':
                    
                    self.cur_tracker.del_item(resp)
                    time.sleep(2)
                    break
                else:
                    continue
            else:
                print("Item not in wishlist. Please try again.")
                
    
    def redeem_item(self):
        
        """
        Runs the redeem function from tracker. Asks user what item to mark as redeemed. 
        If item is in wishlist, then redeem it. Subtract the points and cost from the 
        tracker totals. 
        
        """
        
        while True:
            # TO DO: make it not as case sensitive
            resp = input("What item do you want to redeem from the wishlist? Please type the exact name: ")
            
            if self.cur_tracker.is_in_wishlist(resp):
                
                resp2 = input("Please type Y to confirm that you want to redeem this item: ")
                
                if resp2.lower() == 'y':
                    
                    self.cur_tracker.redeem_item(resp)
                    break
                else:
                    continue
            else:
                print("Item not in wishlist. Please try again.")
    
    def search_item(self):
        
        while True:
            
            resp = input("What is the name of the item you want to search for? ")
            
            if self.cur_tracker.is_in_wishlist(resp):
                
                temp = self.cur_tracker.search_wishlist(resp)
                # print the item info
                
                print("""Item Name: {} \n
                       Category: {} \n
                       Value: {} \n
                       Price: {} \n
                       Date Entered: {} \n
                       Redeemed: {} \n""".format(resp, temp["category"], temp["value"],
                                              temp["price"], temp["date"], temp["redeemed"]))
                break
            else:
                print("Item not in wishlist. Please try again using the exact name of the item.")
                
                
    
    def print_wishlist(self):
        """ 
        Utilizes the print wishlist function the tracker to present 
        either a wishlist of only active items or the full wishlist regardless
        of deletion or redemption
        """
        while True:
            
            resp = input("Do you want to see only active items? Please enter y/n")
            
            if resp.lower()[0] == "y":
                print(self.cur_tracker.print_wishlist())
                break
            elif resp.lower()[0] == "n":
                print(self.cur_tracker.print_wishlist(only_active = 'n'))
                break
            else:
                print("Invalid response. Please try again.")
                
    
    def tracker_info(self, debug = "n"):
        
        """ Displays information about the tracker """
        
        # QUESTION: Update with more metadata like last save?
        
        self.cur_tracker.calc()
        
        while True:
            
            resp = input("View tracker info in debugging mode? y/n: ")
            
            print("""
                {}'s current points are: {}
                {}'s wishlist current point totals up to: {}
                {}'s wishlist current cost totals up to: {:.2f}
                """.format(self.cur_username, self.cur_tracker.points, self.cur_username, self.cur_tracker.wl_points, self.cur_username, self.cur_tracker.cost))
            
            if resp == "y":
                self.cur_tracker.tracker_info(debug = "y")
                break
            elif resp == "n":
                self.cur_tracker.tracker_info(debug = "n")
                break
        
    
    def save_tracker(self):
        """
        Saves the tracker names into a pickle file inside the data directory
        Saves the current tracker into a pickle file inside the data directory
        """
        now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        
        names_file = "data/tracker_names_" + now + ".pickle"
        tracker_file = "data/" + self.cur_username + "_" + now + ".pickle"
        
        with open(names_file, "wb") as f:
            pickle.dump(self.tracker_names, f)
            
        with open(tracker_file, "wb") as f:
            pickle.dump(self.cur_tracker, f)
            
        print("Impulse Spending Tracker successfully saved.")
        
    
    def quit(self):
        
        """Function that saves before exiting if a user session is active"""
        
        if self.cur_username != '':
            self.save_tracker()
            
        print("The program will now exit. Thank you for using the Impulse Tracker!")
        exit(0)
    
    

In [4]:
m = Session()
m.run()
#m.tracker_info()
#m.save_tracker()


                     __             __   ___     __   __   ___       __          __  
            |  |\/| |__) |  | |    /__` |__     /__` |__) |__  |\ | |  \ | |\ | / _` 
            |  |  | |    \__/ |___ .__/ |___    .__/ |    |___ | \| |__/ | | \| \__> 

            ___  __        __        ___  __                                         
             |  |__)  /\  /  ` |__/ |__  |__)                                        
             |  |  \ /~~\ \__, |  \ |___ |  \     
            

            Welcome to Impulse Spending Tracker, where you can create a wishlist and curb
            impulse spending using a point system. The longer items stay un-redeemed on 
            your wishlist, the more points you accumulate. Accumulate enough points and you
            can redeem items from your wishlist. 

            What would you like to do?

            1) Create a new tracker
            2) Load a saved tracker
            3) Quit
            
Enter your response here: 2
Enter th

SystemExit: 0

C:\Users\tkayt\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3327: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [6]:
import tabulate

In [10]:
test = {'Bose Headphones': {'price': 299.0, 'category': 'Electronics', 'value': 700, 'date': datetime.date(2021, 1, 14), 'redeemed_dt': datetime.date(1900, 1, 1), 'redeemed': 'no', 'del_ind': 'no'}, 'Bow Scrunchies': {'price': 6.95, 'category': 'Clothing/Accessories', 'value': 100, 'date': datetime.date(2021, 5, 9), 'redeemed_dt': datetime.date(1900, 1, 1), 'redeemed': 'no', 'del_ind': 'no'}}

headers = ["Item", "Price", "Category", "Value", "Date"]
flat_test = [(k,) + v for k, v in test.items()]

i is 1
we're inside the inner if


In [14]:
from tabulate import tabulate
test = {'Bose Headphones': {'price': 299.0, 'category': 'Electronics', 'value': 700, 'date': datetime.date(2021, 1, 14), 'redeemed_dt': datetime.date(1900, 1, 1), 'redeemed': 'no', 'del_ind': 'no'}, 'Bow Scrunchies': {'price': 6.95, 'category': 'Clothing/Accessories', 'value': 100, 'date': datetime.date(2021, 5, 9), 'redeemed_dt': datetime.date(1900, 1, 1), 'redeemed': 'no', 'del_ind': 'no'}}

flat_test = []

flat_test = [(k, v["date"].strftime("%m/%d/%Y"), v["category"], float(v["price"]), v["value"]) for k, v in test.items()]

#for k, v in test.items():
    
    #flat_test.append((k, v["date"].strftime("%m/%d/%Y"), v["category"], float(v["price"]), v["value"]))

headers = ["Item", "Date", "Category", "Price", "Value"]
print(tabulate(flat_test, headers = headers))

Item             Date        Category                Price    Value
---------------  ----------  --------------------  -------  -------
Bose Headphones  01/14/2021  Electronics            299         700
Bow Scrunchies   05/09/2021  Clothing/Accessories     6.95      100


In [11]:
m = Session()
m.run()
m.save_tracker()


                     __             __   ___     __   __   ___       __          __  
            |  |\/| |__) |  | |    /__` |__     /__` |__) |__  |\ | |  \ | |\ | / _` 
            |  |  | |    \__/ |___ .__/ |___    .__/ |    |___ | \| |__/ | | \| \__> 

            ___  __        __        ___  __                                         
             |  |__)  /\  /  ` |__/ |__  |__)                                        
             |  |  \ /~~\ \__, |  \ |___ |  \     
            

            Welcome to Impulse Spending Tracker, where you can create a wishlist and curb
            impulse spending using a point system. The longer items stay un-redeemed on 
            your wishlist, the more points you accumulate. Accumulate enough points and you
            can redeem items from your wishlist. 

            What would you like to do?

            1) Create a new tracker
            2) Load a saved tracker
            3) Quit
            
Enter your response here: 2
Enter th

SystemExit: 0

In [14]:
m.cur_username

'TK'

In [7]:
if len(os.listdir("data")) > 0:
    pickles = ["data/" + f for f in os.listdir("data") if f[0:13] == "tracker_names"]
    newest = max(pickles, key = os.path.getctime)
    
with open(newest, "rb") as f:
    
    test = pickle.load(f)
    
print(test)
print(newest)
                

{'Eileen', 'TK', 'Elaine'}
data/tracker_names_20210504_221550.pickle


In [28]:
len(["data/" + f for f in os.listdir("data") if f[0:13] == "tracker_names"])


0

In [42]:
f = open("data/tracker_names_30042021_175113.txt", "a")
f.write("Now the file has more content!")
f.close()

In [31]:
t1 = Tracker("TK")

t1.tracker_info()

Wishlist contains: {}
Start Date of Tracker: 2021-04-28
Username is: TK


In [32]:
t1.add_item("Bose headphones", 299.00, "Electronics", 800, "yes", dt.date(2021, 1, 14))
t1.add_item("Gldn Celine earrings", 29.99, "Jewelry", 400, "yes", dt.date(2021, 1, 14))
t1.add_item("Kris Nations Huggie Hoop earrings", 50.00, "Jewelry", 400, "yes", dt.date(2021, 1, 14))
t1.add_item("Gentlewoman's Agreement Ring", 148.00, "Jewelry", 1000, "yes", dt.date(2021, 1, 14))
t1.add_item("Bliss moisturizer", 19.99, "Beauty/Skincare", 200, "yes", dt.date(2021, 4, 9))
t1.add_item("Watermelon face milk", 14.00, "Beauty/Skincare", 200, "yes", dt.date(2021, 4, 9))
t1.add_item("Leggings", 19.99, "Clothes", 200, "yes", dt.date(2021, 3, 25))
t1.add_item("Slippers", 15.99, "Clothes", 200, "yes", dt.date(2021, 4, 2))
t1.add_item("Salt Acid Fat Heat Cookbook", 18.84, "Books", 200, "yes", dt.date(2021, 3, 15))
t1.add_item("Colsie Bralette", 12.99, "Clothes", 200, "yes", dt.date(2021, 3, 28))
t1.add_item("OPI Nail Polish", 9.99, "Beauty/Skincare", 100, "yes", dt.date(2021, 3, 28))
t1.add_item("Pizza Cupcakes", 38.98, "Food", 600, "yes", dt.date(2021, 3, 25))
t1.add_item("Mushroom Jerky", 35.99, "Food", 600, "yes", dt.date(2021, 3, 25))
t1.add_item("Old Navy Embroidered Sweatshirt", 34.99, "Clothes", 500, "yes", dt.date(2021, 4, 11))
t1.add_item("Winky Lux Lip Stain", 15.99, "Beauty/Skincare", 300, "yes", dt.date(2021, 4, 11))
t1.add_item("Colourpop Lip Stain", 8.00, "Beauty/Skincare", 200, "yes", dt.date(2021, 4, 15))
t1.print_wishlist()

Item added to wishlist
Item added to wishlist
Item added to wishlist
Item added to wishlist
Item added to wishlist
Item added to wishlist
Item added to wishlist
Item added to wishlist
Item added to wishlist
Item added to wishlist
Item added to wishlist
Item added to wishlist
Item added to wishlist
Item added to wishlist
Item added to wishlist
Item added to wishlist


,price,category,value,date,redeemed_dt,redeemed,del_ind
Bose headphones,299,Electronics,800,2021-01-14,1900-01-01,no,no
Gldn Celine earrings,29.99,Jewelry,400,2021-01-14,1900-01-01,no,no
Kris Nations Huggie Hoop earrings,50,Jewelry,400,2021-01-14,1900-01-01,no,no
Gentlewoman's Agreement Ring,148,Jewelry,1000,2021-01-14,1900-01-01,no,no
Bliss moisturizer,19.99,Beauty/Skincare,200,2021-04-09,1900-01-01,no,no
Watermelon face milk,14,Beauty/Skincare,200,2021-04-09,1900-01-01,no,no
Leggings,19.99,Clothes,200,2021-03-25,1900-01-01,no,no
Slippers,15.99,Clothes,200,2021-04-02,1900-01-01,no,no
Salt Acid Fat Heat Cookbook,18.84,Books,200,2021-03-15,1900-01-01,no,no
Colsie Bralette,12.99,Clothes,200,2021-03-28,1900-01-01,no,no


In [33]:
p, c = t1.calc()
t1.redeem_item("Watermelon face milk")
t1.redeem_item("Colourpop Lip Stain")
print(t1.points)
print(t1.print_wishlist())

Item marked as redeemed
Item marked as redeemed
335
                                   price         category value        date  \
Bose headphones                      299      Electronics   800  2021-01-14   
Gldn Celine earrings               29.99          Jewelry   400  2021-01-14   
Kris Nations Huggie Hoop earrings     50          Jewelry   400  2021-01-14   
Gentlewoman's Agreement Ring         148          Jewelry  1000  2021-01-14   
Bliss moisturizer                  19.99  Beauty/Skincare   200  2021-04-09   
Leggings                           19.99          Clothes   200  2021-03-25   
Slippers                           15.99          Clothes   200  2021-04-02   
Salt Acid Fat Heat Cookbook        18.84            Books   200  2021-03-15   
Colsie Bralette                    12.99          Clothes   200  2021-03-28   
OPI Nail Polish                     9.99  Beauty/Skincare   100  2021-03-28   
Pizza Cupcakes                     38.98             Food   600  2021-03-25   


In [59]:
print(t1.calc())

(319, 750.7300000000001)


In [ ]:
# starting to rewrite with item class

class Item():
    
    """
    Item object, which has characteristics of the wishlist item such as name, price, point value, etc.
    """
    
    def __init__(self, name, price, value, link = " ", override_dates = "no", date = dt.date(1900, 1, 1)):
        """
        Create object with base characteristics. Allows for overriding date logging, which defaults to
        current date in case we need to do back track entries.
        """
        
        self.name = name
        self.price = price
        self.value = value
        self.link = link
        self.redeemed_date = " "
        self.redeemed = "no"
        
        if override_dates == "no":
            
            self.date = dt.date.today()
            
        elif override_dates == "yes":
            
            self.date = date
            
    def item_update(self, )



In [13]:
i = 1
try:
    
    if i == 0:
        raise Exception("Zero")
    if i > 0:
        raise Exception("Blah")
except :
    
    print("here")

here
